<h1 style='direction : rtl;font-family:Calibri'>
انتخاب کودهای جدید و ساخت شبکه های جدید LSTM بر مبنای اولویت انتخاب کود بر اساس دسترس پذیری
</h1>

<h4 style='direction : rtl;font-family:Calibri'>
برای ساخت مدل بر اساس اولویت انتخاب کود در مجموعه چند کشور نیاز به دیتاست های import,Agriculture,production نیاز داریم.
همچنین بر اساس تحلیل جدید که روی داده ها دوباره انجام دادیم(مطابق مراحل پایین)مجموعه کشورهای Bangladesh', 'Bhutan', 'Brunei Darussalam', 'India', 'Indonesia',
       'Iran (Islamic Republic of)', 'Iraq', 'Israel', 'Japan',
       'Kazakhstan', 'Kyrgyzstan', 'Maldives', 'Nepal', 'Pakistan',
       'Philippines', 'Republic of Korea', 'Saudi Arabia', 'Turkey' انتخاب شدند.
4 کود که بیشترین تولید و مصرف کشاورزی را در میان سایر کودهای این کشورها داشتند گزینش کردیم.
شامل: و'others'وSuperphosphates above 35%'و'Superphosphatesو 'Urea'
<br>
مدلسازی برای انتخاب دسترس پذیری در هر کشور بر اساس کشورها و کودهای ذکر شده در بالا صورت میگیرد.
</h4>
    

---

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import os
os.chdir('c:\\Users\\ASUS\\Projects\\CI')

In [5]:
df = pd.read_csv('Asia.csv',encoding='latin-1',index_col='Year')
df = df.drop('Flag',axis=1)

In [6]:
df.head()

,Area,Item,Element,Value
Year,,,,
2011,Afghanistan,"Ammonia, anhydrous",Import Quantity,4.49
2014,Afghanistan,"Ammonia, anhydrous",Import Quantity,2.00
2017,Afghanistan,"Ammonia, anhydrous",Import Quantity,2.07
2011,Afghanistan,"Ammonia, anhydrous",Import Value,2.69
2014,Afghanistan,"Ammonia, anhydrous",Import Value,1.07


<h4 style='direction : rtl;font-family:Calibri'>
با فرض اینکه در دسترس ترین کود، کودیست که در کشور تولید و مهم ترین کود کودیست که در کشاورزی مصرف میشود، این دو فاکتور را روی دیتاهای موجود اعمال میکنیم تا کشوری با دیتای غنی تر را بیابیم.
</h4>

In [47]:
AsiaProd = pd.read_csv('Asia_Production.csv',index_col='Year')

In [48]:
AsiaProd.head()

,Area,Item,Value
Year,,,
2013,Afghanistan,Diammonium phosphate (DAP),3346.0
2014,Afghanistan,Diammonium phosphate (DAP),3346.0
2015,Afghanistan,Diammonium phosphate (DAP),3346.0
2016,Afghanistan,Diammonium phosphate (DAP),3346.0
2002,Afghanistan,Urea,50900.0


In [9]:
AsiaProd['Progress'] = AsiaProd['Value'] - AsiaProd['Value'].shift(1)
AsiaProd.head()

,Area,Item,Value,Progress
Year,,,,
2013,Afghanistan,Diammonium phosphate (DAP),3346.0,NaN
2014,Afghanistan,Diammonium phosphate (DAP),3346.0,0.0
2015,Afghanistan,Diammonium phosphate (DAP),3346.0,0.0
2016,Afghanistan,Diammonium phosphate (DAP),3346.0,0.0
2002,Afghanistan,Urea,50900.0,47554.0


In [12]:
AsiaProdVal = pd.DataFrame(AsiaProd.groupby(by=['Area','Item']).mean()['Progress'])
AsiaProdVal

Progress
Area        Item                                                    
Afghanistan Diammonium phosphate (DAP)                      0.000000
            Urea                                         1866.266667
Bahrain     Urea                                        44177.333333
Bangladesh  Ammonium sulphate                         -137800.000000
            Diammonium phosphate (DAP)                  10777.777778
...                                                              ...
Uzbekistan  Urea and ammonium nitrate solutions (UAN)  -99064.333333
Viet Nam    Fertilizers n.e.c.                         102351.000000
            NPK fertilizers                            394337.142857
            Superphosphates above 35%                 -138262.500000
            Urea                                        68493.750000

[277 rows x 1 columns]

In [11]:
#AsiaProdVal = AsiaProdVal.reset_index()

In [13]:
candidate_countries = []
candidate_fertilizer = []
count = 0
for i in range(0,273):
    while AsiaProdVal.iloc[i].name[0] == AsiaProdVal.iloc[i+1].name[0]:
        count = count +1
        i =i +1
    i = i-1
    if(count > 4 ):
        candidate_countries.append(AsiaProdVal.iloc[i].name[0])
        candidate_fertilizer.append(AsiaProdVal.iloc[i].name[1])

    count = 0

In [14]:

np.unique(candidate_countries)

array(['Bangladesh', 'Bhutan', 'Brunei Darussalam', 'India', 'Indonesia',
       'Iran (Islamic Republic of)', 'Iraq', 'Israel', 'Japan',
       'Kazakhstan', 'Kyrgyzstan', 'Maldives', 'Nepal', 'Pakistan',
       'Philippines', 'Republic of Korea', 'Russian Federation',
       'Saudi Arabia', 'Turkey', 'Uzbekistan'], dtype='<U26')

In [15]:
np.unique(candidate_fertilizer)

array(['Phosphate rock', 'Superphosphates above 35%',
       'Superphosphates, other', 'Urea'], dtype='<U25')

<h4 style='direction : rtl;font-family:Calibri'>
بخش کشاورزی
</h4>

In [46]:
AsiaAgri = pd.read_csv('Asia - Agriculture.csv',index_col='Year')
AsiaAgri = AsiaAgri.drop(['Area Code','Item Code','Element Code','Unit','Flag','continent','Element','Year Code'],axis=1)
#AsiaAgri.set_index('Year Code')

In [17]:
AsiaAgri.head()

,Area,Item,Value
Year,,,
2002,Afghanistan,NPK fertilizers,17900.0
2003,Afghanistan,NPK fertilizers,33200.0
2004,Afghanistan,NPK fertilizers,47700.0
2004,Afghanistan,Urea,42300.0
2005,Afghanistan,Urea,20577.0


In [18]:
AsiaAgri['Progress'] = AsiaAgri['Value'] - AsiaAgri['Value'].shift(1)
AsiaAgri.head()

,Area,Item,Value,Progress
Year,,,,
2002,Afghanistan,NPK fertilizers,17900.0,NaN
2003,Afghanistan,NPK fertilizers,33200.0,15300.0
2004,Afghanistan,NPK fertilizers,47700.0,14500.0
2004,Afghanistan,Urea,42300.0,-5400.0
2005,Afghanistan,Urea,20577.0,-21723.0


In [19]:
AsiaAgri = pd.DataFrame(AsiaAgri.groupby(by=['Area','Item']).mean()['Progress'])
AsiaAgri

Progress
Area        Item                                                            
Afghanistan NPK fertilizers                                     14900.000000
            Urea                                                 7441.000000
Armenia     Ammonium nitrate (AN)                               -7358.100000
            Ammonium sulphate                                   -3364.200000
            Calcium ammonium nitrate (CAN) and other mixtur...     39.800000
...                                                                      ...
Yemen       Other NP compounds                                     27.250000
            Potassium chloride (muriate of potash) (MOP)          111.000000
            Superphosphates above 35%                            -304.666667
            Urea                                                 4273.428571
            Urea and ammonium nitrate solutions (UAN)           -7494.000000

[499 rows x 1 columns]

In [ ]:
#AsiaAgri = AsiaAgri.reset_index()

In [20]:
candidateAgri_countries = []
candidateAgri_fertilizer = []
count = 0
for i in range(0,490):
    while AsiaAgri.iloc[i].name[0] == AsiaAgri.iloc[i+1].name[0]:
        count = count +1
        i =i +1
    i = i-1
    if(count > 4 ):
        candidateAgri_countries.append(AsiaAgri.iloc[i].name[0])
        candidateAgri_fertilizer.append(AsiaAgri.iloc[i].name[1])

    count = 0

In [21]:

np.unique(candidateAgri_countries)

array(['Armenia', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei Darussalam',
       'Cambodia', 'China, Hong Kong SAR', 'China, Macao SAR',
       'China, Taiwan Province of', 'China, mainland', 'India',
       'Indonesia', 'Iran (Islamic Republic of)', 'Iraq', 'Israel',
       'Japan', 'Jordan', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Lebanon',
       'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'Oman',
       'Pakistan', 'Philippines', 'Republic of Korea', 'Saudi Arabia',
       'Singapore', 'Sri Lanka', 'Syrian Arab Republic', 'Tajikistan',
       'Thailand', 'Turkey', 'Viet Nam'], dtype='<U26')

In [22]:
np.unique(candidateAgri_fertilizer)

array(['Superphosphates above 35%', 'Superphosphates, other', 'Urea'],
      dtype='<U25')

In [23]:
np.intersect1d(np.unique(candidate_countries),np.unique(candidateAgri_countries))

array(['Bangladesh', 'Bhutan', 'Brunei Darussalam', 'India', 'Indonesia',
       'Iran (Islamic Republic of)', 'Iraq', 'Israel', 'Japan',
       'Kazakhstan', 'Kyrgyzstan', 'Maldives', 'Nepal', 'Pakistan',
       'Philippines', 'Republic of Korea', 'Saudi Arabia', 'Turkey'],
      dtype='<U26')

<h4 style='direction : rtl;font-family:Calibri'>
دیتاهای جدید را بر اساس کشور و کود انتخاب شده پردازش میکنیم.
</h4>

In [56]:
countries = ['Bangladesh', 'Bhutan', 'Brunei Darussalam', 'India', 'Indonesia',
       'Iran (Islamic Republic of)', 'Iraq', 'Israel', 'Japan',
       'Kazakhstan', 'Kyrgyzstan', 'Maldives', 'Nepal', 'Pakistan',
       'Philippines', 'Republic of Korea', 'Saudi Arabia', 'Turkey']

AsiaAgriUrea = AsiaAgri[AsiaAgri['Item']=='Urea']
AsiaAgriUrea = AsiaAgriUrea.drop('Item',axis=1)
AsiaAgriUrea = AsiaAgriUrea.pivot_table(columns='Area',values='Value')
AsiaAgriUrea = AsiaAgriUrea[countries]

AsiaAgriSupA = AsiaAgri[AsiaAgri['Item']=='Superphosphates above 35%']
AsiaAgriSupA = AsiaAgriSupA.drop('Item',axis=1)
AsiaAgriSupA=AsiaAgriSupA.pivot_table(columns='Area',values='Value')
AsiaAgriSupA = AsiaAgriSupA[countries != ['Israel', 'Philippines', 'Bhutan', 'Republic of Korea']]

AsiaAgriSup = AsiaAgri[AsiaAgri['Item']=='Superphosphates, other']
AsiaAgriSup = AsiaAgriSup.drop('Item',axis=1)
AsiaAgriSup=AsiaAgriSup.pivot_table(columns='Area',values='Value')
AsiaAgriSup = AsiaAgriSup[countries]


KeyError: True

In [44]:
AsiaProdUrea = AsiaProd[AsiaProd['Item']=='Urea']
AsiaProdUrea = AsiaProdUrea.drop('Item',axis=1)
AsiaProdUrea=AsiaProdUrea.pivot_table(columns='Area',values='Value')
AsiaProdUrea = AsiaProdUrea[countries]

AsiaProdSupA = AsiaProd[AsiaProd['Item']=='Superphosphates above 35%']
AsiaProdSupA = AsiaProdSupA.drop('Item',axis=1)
AsiaProdSupA=AsiaProdSupA.pivot_table(columns='Area',values='Value')
AsiaProdSupA = AsiaProdSupA[countries]

AsiaProdSup = AsiaProd[AsiaProd['Item']=='Superphosphates, other']
AsiaProdSup = AsiaProdSup.drop('Item',axis=1)
AsiaProdSup=AsiaProdSup.pivot_table(columns='Area',values='Value')
AsiaProdSup = AsiaProdSup[countries]

AsiaProdOther =AsiaProd[AsiaProd['Item']=='other']
AsiaProdOther = AsiaProdOther.drop('Item',axis=1)
AsiaProdOther=AsiaProdOther.pivot_table(columns='Area',values='Value')
AsiaProdOther = AsiaProdOther[countries]


AsiaExp = pd.read_csv('Asia - Export Quantity.csv',index_col='Year')
AsiaExp = AsiaExp.drop(['Area Code','Item Code','Element Code','Unit','Flag','continent','Element','Year Code'],axis=1)

AsiaExpUrea = AsiaExp[AsiaExp['Item']=='Urea']
AsiaExpUrea=AsiaExpUrea.drop('Item',axis=1)
AsiaExpUrea=AsiaExpUrea.pivot_table(columns='Area',values='Value')
AsiaExpUrea = AsiaExpUrea[countries]

AsiaExpSupA = AsiaExp[AsiaExp['Item']=='Superphosphates above 35%']
AsiaExpSupA=AsiaExpSupA.drop('Item',axis=1)
AsiaExpSupA=AsiaExpSupA.pivot_table(columns='Area',values='Value')
AsiaExpSupA = AsiaExpSupA[countries]

AsiaExpSup = AsiaExp[AsiaExp['Item']=='Superphosphates, other']
AsiaExpSup = AsiaExpSup.drop('Item',axis=1)
AsiaExpSup=AsiaExpSup.pivot_table(columns='Area',values='Value')
AsiaExpSup = AsiaExpSup[countries]



KeyError: "['Philippines', 'Israel', 'Saudi Arabia', 'Republic of Korea'] not in index"

In [ ]:
# Asia_importQ_Urea.isnull().sum()
#
#

In [ ]:
# removed_countries = []
# for (columnName,columnData ) in Asia_importQ_Urea.iteritems():
#     if (columnData.isnull().sum()) >= 3:
#         removed_countries.append(columnName)
#

In [ ]:
# removed_countries
#

Handling two countries with most missing values.

In [ ]:
# Asia_importQ_Urea = Asia_importQ_Urea.drop(removed_countries,axis=1)
#

In [ ]:
# Asia_importQ_Urea['Uzbekistan'].plot()
#
#

In [ ]:
# Asia_importQ_Urea['Tajikistan'].plot()
#

In [ ]:
# Asia_importQ_Urea['Tajikistan'].fillna(0,inplace=True)
# for (columnName,columnData ) in Asia_importQ_Urea.iteritems():
#     Asia_importQ_Urea[columnName].fillna(columnData.mean(),inplace=True)
#

In [ ]:
# Asia_importQ_Urea.head()
#

In [ ]:
# Asia_importQ_Urea.to_csv('Processed_data.csv')